# Bab 8 Backpropagation (2)

## Praktikum

### a) Fungsi *Sigmoid*, *One-Hot Encoding* dan *Decoding*

In [ ]:
import numpy as np

# Fungsi sigmoid dan turunannya
def sig(x):
    return 1 / (1 + np.exp(-x))

def sigd(x):
    return sig(x) * (1 - sig(x))

# Fungsi one-hot encoding dan decoding
def onehot_enc(target):
    return np.eye(len(np.unique(target)))[target]

def onehot_dec(encoded):
    return np.argmax(encoded, axis=1)

### b) Fungsi *Training* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
def bp_fit(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
    np.random.seed(1)
    nin = [np.empty(i) for i in layer_conf]
    n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
    w = [np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)]
    dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]
    epoch = 0
    mse = 1

    for i in range(0, len(n)-1):
        n[i][-1] = 1

    while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0

        for r in range(len(X)):
            n[0][:-1] = X[r]

            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L-1], w[L-1])
                n[L][:len(nin[L])] = sig(nin[L])

            e = target[r] - n[-1]
            mse += sum(e ** 2)
            d[-1] = e * sigd(nin[-1])
            dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))

            for L in range(len(layer_conf) - 1, 1, -1):
                din[L-2] = np.dot(d[L-1], np.transpose(w[L-1][:-1]))
                d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
                dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1, 1))

            # Update weights
            for i in range(len(w)):
                w[i] += dw[i]

        mse /= len(X)

        if print_per_epoch > -1 and epoch % print_per_epoch == 0:
            print(f'Epoch {epoch}, MSE: {mse}')

    return w, epoch, mse

### c) Fungsi *Testing* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
def bp_predict(X, w):
  n = [np.empty(len(i)) for i in w]
  nin = [np.empty(len(i[0])) for i in w]
  predict = []

  n.append(np.empty(len(w[-1][0])))

  for x in X:
    n[0][:-1] = x

    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L + 1][:len(nin[L])] = sig(nin[L])

    predict.append(n[-1].copy())

  return predict

### d) Percobaan Klasifikasi Dataset Iris

![Iris Dataset](https://www.spataru.at/images/blog/iris-dataset-svm/iris_types.jpg)

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 3, 3), learn_rate=0.1, max_epoch=1000, max_error=0.1, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epoch 25, MSE: 0.4573000553790559
Epoch 50, MSE: 0.321272689922169
Epoch 75, MSE: 0.2668003450939322
Epoch 100, MSE: 0.19045841193641896
Epoch 125, MSE: 0.13206064032817524
Epoch 150, MSE: 0.10002434429710472
Epochs: 151, MSE: 0.09910797309769231
Output: [0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 2 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 2 2 1]
True : [0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 2 2 1]
Accuracy: 0.9777777777777777


# **Soal**

Nomor 1

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 2, 3), learn_rate=0.1, max_epoch=100, max_error=0.5, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

# Testing on training data
train_predict = bp_predict(X_train, w)
train_predict = onehot_dec(train_predict)
y_train_dec = onehot_dec(y_train)
train_accuracy = accuracy_score(train_predict, y_train_dec)

print('Train Output:', train_predict)
print('Train True :', y_train_dec)
print('Train Accuracy:', train_accuracy)

# Testing on test data
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test_dec = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test_dec)

print('Test Output:', predict)
print('Test True :', y_test_dec)
print('Test Accuracy:', accuracy)

Epoch 25, MSE: 0.5587148548510855
Epochs: 29, MSE: 0.49569003845261594
Train Output: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Train True : [2 0 0 0 1 0 0 2 2 2 2 2 1 2 1 0 2 2 0 0 2 0 2 2 1 1 2 2 0 1 1 2 1 2 1 0 0
 0 2 0 1 2 2 0 0 1 0 2 1 2 2 1 2 2 1 0 1 0 1 1 0 1 0 0 2 2 2 0 0 1 0 2 0 2
 2 0 2 0 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 2 0 0 2 1 2 1 2 2 1 2 0]
Train Accuracy: 0.3523809523809524
Test Output: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2]
Test True : [0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 2 2 1]
Test Accuracy: 0.28888888888888886


Nomor 2

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 25, 3), learn_rate=0.1, max_epoch=10000, max_error=0.01, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

# Testing on training data
train_predict = bp_predict(X_train, w)
train_predict = onehot_dec(train_predict)
y_train_dec = onehot_dec(y_train)
train_accuracy = accuracy_score(train_predict, y_train_dec)

print('Train Output:', train_predict)
print('Train True :', y_train_dec)
print('Train Accuracy:', train_accuracy)

# Testing on test data
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test_dec = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test_dec)

print('Test Output:', predict)
print('Test True :', y_test_dec)
print('Test Accuracy:', accuracy)

Epoch 25, MSE: 1.9997475602526067
Epoch 50, MSE: 1.9996839107720699
Epoch 75, MSE: 1.9995729256252905
Epoch 100, MSE: 1.9993234166160632
Epoch 125, MSE: 1.998004842894972
Epoch 150, MSE: 1.5034310711672474
Epoch 175, MSE: 0.9882734055300789
Epoch 200, MSE: 0.9358385629366138
Epoch 225, MSE: 0.8962403847767554
Epoch 250, MSE: 0.5720113088188044
Epoch 275, MSE: 0.1718964155883048
Epoch 300, MSE: 0.13551945448567226
Epoch 325, MSE: 0.10976277201695424
Epoch 350, MSE: 0.09159154648169049
Epoch 375, MSE: 0.07929291362007068
Epoch 400, MSE: 0.07095077966556529
Epoch 425, MSE: 0.06511610472051664
Epoch 450, MSE: 0.06088086243803241
Epoch 475, MSE: 0.05769828099112584
Epoch 500, MSE: 0.05523313404391771
Epoch 525, MSE: 0.05327321999520479
Epoch 550, MSE: 0.051680019060663
Epoch 575, MSE: 0.05036062409674887
Epoch 600, MSE: 0.0492511427801703
Epoch 625, MSE: 0.04830653743243751
Epoch 650, MSE: 0.0474942479018858
Epoch 675, MSE: 0.0467901200965703
Epoch 700, MSE: 0.046175774075554316
Epoch 725, 

Nomor 3

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 4, 3), learn_rate=0.1, max_epoch=850, max_error=0.01, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

# Testing on training data
train_predict = bp_predict(X_train, w)
train_predict = onehot_dec(train_predict)
y_train_dec = onehot_dec(y_train)
train_accuracy = accuracy_score(train_predict, y_train_dec)

print('Train Output:', train_predict)
print('Train True :', y_train_dec)
print('Train Accuracy:', train_accuracy)

# Testing on test data
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test_dec = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test_dec)

print('Test Output:', predict)
print('Test True :', y_test_dec)
print('Test Accuracy:', accuracy)

Epoch 25, MSE: 0.4037189784961099
Epoch 50, MSE: 0.30392164211321915
Epoch 75, MSE: 0.2467626574560707
Epoch 100, MSE: 0.18371215918575698
Epoch 125, MSE: 0.13722833341171944
Epoch 150, MSE: 0.10766703344361239
Epoch 175, MSE: 0.0891059387107455
Epoch 200, MSE: 0.07720900091073199
Epoch 225, MSE: 0.06928833189909853
Epoch 250, MSE: 0.06377510363938882
Epoch 275, MSE: 0.05976793063739653
Epoch 300, MSE: 0.056741123292045205
Epoch 325, MSE: 0.05437885321180183
Epoch 350, MSE: 0.052484652174752
Epoch 375, MSE: 0.05093188930466216
Epoch 400, MSE: 0.04963608911914036
Epoch 425, MSE: 0.048539022400874925
Epoch 450, MSE: 0.04759929543082216
Epoch 475, MSE: 0.04678662643979523
Epoch 500, MSE: 0.04607827250402464
Epoch 525, MSE: 0.045456743342417276
Epoch 550, MSE: 0.04490830362959628
Epoch 575, MSE: 0.04442196901975075
Epoch 600, MSE: 0.04398881760969524
Epoch 625, MSE: 0.04360150689804629
Epoch 650, MSE: 0.04325392722033676
Epoch 675, MSE: 0.04294094760595845
Epoch 700, MSE: 0.042658225482258